[Motion Clouds](https://neuralensemble.github.io/MotionClouds) were defined in the origin to define parameterized moving textures. In [that other post](https://laurentperrinet.github.io/sciblog/posts/2016-07-14_static-motion-clouds.html), we defined a simple code to generate static images using a simple code. Can we generate a series of images while chanigning the phase *globally*?

<!-- TEASER_END -->

There are multiple solutions, and the simplest is perhaps to generate a movie that does not move. This means that the mean velocity $(V_X, V_Y)$ is null $=(0, 0)$ but also that there is no noise in the definition of the speed plane. This is defined by $B_V=0$, that is, that all the energy is concentrated on the spped plane:

In [ ]:
from __future__ import division, print_function
import numpy as np
np.set_printoptions(precision=6, suppress=True)
import os
%matplotlib inline
#%config InlineBackend.figure_format='retina'
%config InlineBackend.figure_format = 'svg'
import matplotlib.pyplot as plt
phi = (np.sqrt(5)+1)/2
fig_width = 10
figsize = (fig_width, fig_width/phi)
from IPython.display import display, HTML
def show_video(filename): 
    return HTML(data='<video src="{}" loop autoplay width="600" height="600"></video>'.format(filename))
%load_ext autoreload
%autoreload 2

## some book keeping for the notebook

In [1]:
%load_ext watermark
%watermark

2017-10-06T11:01:51+02:00

CPython 3.6.3
IPython 6.2.1

compiler   : GCC 4.2.1 Compatible Apple LLVM 9.0.0 (clang-900.0.37)
system     : Darwin
release    : 17.0.0
machine    : x86_64
processor  : i386
CPU cores  : 8
interpreter: 64bit


In [2]:
%load_ext version_information
%version_information numpy, scipy, matplotlib, sympy, pillow, imageio

Software versions
Python 3.6.3 64bit [GCC 4.2.1 Compatible Apple LLVM 9.0.0 (clang-900.0.37)]
IPython 6.2.1
OS Darwin 17.0.0 x86_64 i386 64bit
numpy 1.13.3
scipy 0.19.1
matplotlib 2.1.0rc1
sympy 1.1.1
pillow 4.3.0
imageio 2.1.2
Fri Oct 06 11:01:55 2017 CEST

In [ ]:
%%bash
cd ..
nikola build ; nikola deploy
cd posts